# FMU ME: Van der Pol

This example demonstrates **Model Exchange FMU** integration with a nonlinear oscillator. The Van der Pol equation exhibits self-sustained oscillations:


$$\ddot{x} - \mu(1 - x^2)\dot{x} + x = 0$$


where $\mu > 0$ controls nonlinearity. As a first-order system:

$$\frac{dx_0}{dt} = x_1$$
$$\frac{dx_1}{dt} = \mu(1 - x_0^2)x_1 - x_0$$

You can also find the FMU integration tests in the [GitHub repository](https://github.com/milanofthe/pathsim/blob/master/tests/evals/).

## Import and Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Apply PathSim docs matplotlib style for consistent, theme-friendly figures
plt.style.use('../pathsim_docs.mplstyle')

from pathlib import Path
from pathsim import Simulation, Connection
from pathsim.blocks import ModelExchangeFMU, Scope
from pathsim.solvers import RKDP54, ESDIRK43

## FMU Path

In [ ]:
notebook_dir = Path().resolve()
fmu_path = notebook_dir / "data" / "VanDerPol_ME.fmu"

# Verify FMU exists
if not fmu_path.exists():
    raise FileNotFoundError(f"FMU file not found at {fmu_path}")

## System Definition

We simulate with $\mu = 1.0$ for clear nonlinear oscillations:

In [ ]:
# Create Model Exchange FMU
fmu = ModelExchangeFMU(
    fmu_path=str(fmu_path),
    instance_name="vanderpol",
    start_values={
        "mu": 1.0,     # nonlinearity parameter
        "x0": 2.0,     # initial position
        "x1": 0.0,     # initial velocity
    },
    tolerance=1e-8,
    verbose=False
)

# Scope to record states
sco = Scope(labels=[r"$x_0$", r"$x_1$"])

blocks = [fmu, sco]

# Connections
connections = [
    Connection(fmu[0], sco[0]),
    Connection(fmu[1], sco[1]),
]

Display FMU metadata:

In [ ]:
# Display FMU metadata (accessed via fmu_wrapper)
md = fmu.fmu_wrapper.model_description
print(f"Model Name: {md.modelName}")
print(f"FMI Version: {fmu.fmu_wrapper.fmi_version}")
print(f"Description: {md.description}")
print(f"Generation Tool: {md.generationTool}")
print(f"\nContinuous states: {fmu.fmu_wrapper.n_states}")
print(f"Event indicators: {fmu.fmu_wrapper.n_event_indicators}")
print(f"Outputs: {len(fmu.fmu_wrapper.output_refs)}")

## Simulation Setup

In [ ]:
# Initialize simulation
sim = Simulation(
    blocks,
    connections,
    dt=0.1,
    dt_max=0.1,
    Solver=RKDP54,
    tolerance_lte_rel=1e-6,
    tolerance_lte_abs=1e-9,
    log=True
)

# Run simulation
sim.run(30.0)

## Results

In [ ]:
sco.plot()
plt.show()

## Relaxation Oscillations

For very large $\mu$, the system exhibits relaxation oscillations with extreme stiffness. We use $\mu = 500$ to demonstrate PathSim's capability with severe stiffness:

In [ ]:
fmu_stiff = ModelExchangeFMU(
    fmu_path=str(fmu_path),
    instance_name="vdp_stiff",
    start_values={"mu": 500.0, "x0": 2.0, "x1": 0.0},
    tolerance=1e-8,
    verbose=False
)

sco_stiff = Scope(labels=[r"$x_0$", r"$x_1$"])

sim_stiff = Simulation(
    [fmu_stiff, sco_stiff],
    [Connection(fmu_stiff[0], sco_stiff[0]), Connection(fmu_stiff[1], sco_stiff[1])],
    dt=0.1,
    Solver=ESDIRK43,          # implicit solver for stiff systems
    tolerance_lte_rel=1e-4,
    tolerance_lte_abs=1e-6,
    tolerance_fpi=1e-8,
    log=True
)

sim_stiff.run(1500.0)

In [ ]:
time_stiff, (x0_stiff, x1_stiff) = sco_stiff.read()

fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True, dpi=130)

axes[0].plot(time_stiff, x0_stiff, lw=2)
axes[0].set_ylabel(r'$x_0$')
axes[0].set_title(r'Relaxation Oscillations ($\mu = 500$)')
axes[0].grid(True, alpha=0.3)

axes[1].plot(time_stiff, x1_stiff, lw=2, color='orange')
axes[1].set_xlabel('Time [s]')
axes[1].set_ylabel(r'$x_1$')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Key Features

- Seamless integration of nonlinear dynamics
- Adaptive timestepping for varying dynamics  
- Parameter sweep capabilities
- Handles moderately stiff systems